In [1]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras 
from keras.models import Sequential
from keras.layers import Dense
from keras.models import model_from_json
from skimage.transform import resize

Using TensorFlow backend.


In [8]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename): 
    print('File {}'.format(filename))
    ds = pydicom.dcmread(filename)
    
    if (ds.BodyPartExamined !='CHEST') | (ds.Modality !='DX') | (ds.PatientPosition not in ['PA', 'AP']):
        print('The image is not valid because the image position, the image type or the body part is not as per standards')
        return
    else:
        print('ID:', ds.PatientID,
             'Age:',ds.PatientAge,
             'Modality:', ds.Modality,
             'Position:', ds.PatientPosition,
             'Body Part:', ds.BodyPartExamined,
             'Study Desc:', ds.StudyDescription)
        
    img = ds.pixel_array    
    return img
    
    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img,img_mean,img_std,img_size): 
        img = resize(img,(224,224))
        img = img/255.0
        grey_img = (img - img_mean) / img_std
        
        proc_img = np.zeros((224,224,3))
        proc_img[:,:, 0] = grey_img
        proc_img[:,:, 1] = grey_img
        proc_img[:,:, 2] = grey_img
        
        proc_img = np.resize(proc_img , img_size)
    
        return proc_img

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
        json_file = open(model_path , 'r')
        loaded_model_json = json_file.read()
        json_file.close()
        model = model_from_json(loaded_model_json)
        model.load_weights(weight_path)
        return model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh):
        result = model.predict(img)
        print('Predicted value:', result)
        
        predict = result[0]
        prediction = "Negative"
        if(predict > thresh):
            prediction = "Positive"
    
        return prediction 

In [10]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = "my_model.json" #path to saved model
weight_path = "xray_class_my_model.best.hdf5"#path to saved best weights

IMG_SIZE=(1,224,224,3) # This might be different if you did not use vgg16
img_mean = 0.510408# loads the mean image value they used during training preprocessing
img_std = 0.25040054# loads the std dev image value they used during training preprocessing

my_model = load_model(model_path, weight_path)#loads model
thresh = 0.4328137#loads the threshold they chose for model classification 

# use the .dcm files to test your prediction
for i in test_dicoms:
    
    img = np.array([])
    img = check_dicom(i)
    
    if img is None:
        continue
        
    img_proc = preprocess_image(img,img_mean,img_std,IMG_SIZE)
    pred = predict_image(my_model,img_proc,thresh)
    print('Model Classification:', pred , 'for Pneumonia' )


File test1.dcm
ID: 2 Age: 81 Modality: DX Position: PA Body Part: CHEST Study Desc: No Finding
Predicted value: [[0.45847535]]
Model Classification: Positive for Pneumonia
File test2.dcm
ID: 1 Age: 58 Modality: DX Position: AP Body Part: CHEST Study Desc: Cardiomegaly
Predicted value: [[0.45849732]]
Model Classification: Positive for Pneumonia
File test3.dcm
ID: 61 Age: 77 Modality: DX Position: AP Body Part: CHEST Study Desc: Effusion
Predicted value: [[0.45880696]]
Model Classification: Positive for Pneumonia
File test4.dcm
The image is not valid because the image position, the image type or the body part is not as per standards
File test5.dcm
The image is not valid because the image position, the image type or the body part is not as per standards
File test6.dcm
The image is not valid because the image position, the image type or the body part is not as per standards
